In [1]:
import os
import requests
from dotenv import load_dotenv
from typing import Any, Type, List
from langchain_core.tools import BaseTool
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain import hub
from langchain_openai import AzureChatOpenAI
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain.memory import ConversationBufferMemory
from datetime import datetime

load_dotenv()
CLICKUP_TOKEN = os.getenv("CLICKUP_TOKEN")

In [2]:
default_headers = {"Authorization": f"{CLICKUP_TOKEN}"}
bearer_headers = {"Authorization": f"Bearer {CLICKUP_TOKEN}"}

## Create Task Tool

In [3]:
from composio.tools.local.clickup.actions.create_task import CreateTask, CreateTaskRequest, CreateTaskResponse

class CreateTaskTool(BaseTool):
    name: str = "create_task_tool"
    description: str = """
    Ferramenta para criar uma nova tarefa no ClickUp com base nos parâmetros fornecidos.
    - Criar Tarefa:
        Invocar: "CreateTaskTool" com os parâmetros apropriados.
    """
    args_schema: Type[BaseModel] = CreateTaskRequest
    headers: dict = {"Authorization": f"{CLICKUP_TOKEN}"}

    def __init__(self, **data):
        super().__init__(**data)

    def _run(self, list_id: int, **task_data) -> Any:
        """Executa a criação de tarefa no ClickUp"""

        action = CreateTask()

        url = f"{action.url}{action.path}".format(list_id=list_id)
        params = {key: value for key, value in task_data.items() if value is not None}

        response = requests.post(url, headers=self.headers, json=params)

        if response.status_code == 201:
            response_json = response.json()
        else:
            try:
                response_json = response.json()
            except requests.JSONDecodeError:
                response_json = {"error": "Invalid JSON response"}

        print(f"Response Body: {response_json}")
        return CreateTaskResponse(data=response_json)

/Users/morossini/anaconda3/envs/secretary/lib/python3.11/site-packages/paramiko/pkey.py:100: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "cipher": algorithms.TripleDES,
/Users/morossini/anaconda3/envs/secretary/lib/python3.11/site-packages/paramiko/transport.py:259: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "class": algorithms.TripleDES,


## Delete Task Tool

In [4]:
from composio.tools.local.clickup.actions.delete_task import DeleteTask, DeleteTaskRequest, DeleteTaskResponse

class DeleteTaskTool(BaseTool):
    name: str = "delete_task_tool"
    description: str = """
    Ferramenta para deletar uma tarefa no ClickUp com base nos parâmetros fornecidos.
    - Deletar Tarefa:
        Invocar: "DeleteTaskTool" com os parâmetros apropriados.
    """
    args_schema: Type[BaseModel] = DeleteTaskRequest
    headers: dict = {"Authorization": f"Bearer {CLICKUP_TOKEN}"}

    def __init__(self, **data):
        super().__init__(**data)

    def _run(self, task_id: str, **delete_params) -> Any:
        """Executa a deleção de tarefa no ClickUp"""

        action = DeleteTask()


        url = f"{action.url}{action.path}".format(task_id=task_id)
        params = {key: value for key, value in delete_params.items() if value is not None}

        response = requests.delete(url, headers=self.headers, params=params)

        if response.status_code == 204:
            response_json = {"message": "Task deleted successfully"}
        else:
            try:
                response_json = response.json()
            except requests.JSONDecodeError:
                response_json = {"error": "Invalid JSON response"}
        
        print(f"Response Body: {response_json}")
        return DeleteTaskResponse(data=response_json)

## Update Task Tool

In [5]:
from composio.tools.local.clickup.actions.update_task import UpdateTask, UpdateTaskRequest, UpdateTaskResponse

class UpdateTaskRequestCustom(UpdateTaskRequest):
    assignees_add: List[int] = Field(
        default=None,
        alias="assignees__add",
        description="",
    )
    assignees_rem: List[int] = Field(
        default=None,
        alias="assignees__rem",
        description="",
    )


class UpdateTaskTool(BaseTool):
    name: str = "update_task_tool"
    description: str = """
    Ferramenta para atualizar uma tarefa no ClickUp com base nos parâmetros fornecidos.
    - Atualizar Tarefa:
        Invocar: "UpdateTaskTool" com os parâmetros apropriados.
    """
    args_schema: Type[BaseModel] = UpdateTaskRequestCustom
    headers: dict = {"Authorization": f"{CLICKUP_TOKEN}"}

    def __init__(self, **data):
        super().__init__(**data)

    def _run(self, task_id: str, **update_params) -> Any:
        """Executa a atualização de tarefa no ClickUp"""

        action = UpdateTask()

        url = f"{action.url}{action.path}".format(task_id=task_id)
        params = {key: value for key, value in update_params.items() if value is not None}

        response = requests.put(url, headers=self.headers, json=params)

        if response.status_code == 200:
            response_json = response.json()
        else:
            try:
                response_json = response.json()
            except requests.JSONDecodeError:
                response_json = {"error": "Invalid JSON response"}
        
        print(f"Response Body: {response_json}")
        return UpdateTaskResponse(data=response_json)

## Create Task Comment

In [6]:
from composio.tools.local.clickup.actions.create_task_comment import CreateTaskComment, CreateTaskCommentRequest, CreateTaskCommentResponse

class CreateTaskCommentTool(BaseTool):
    name: str = "create_task_comment_tool"
    description: str = """
    Ferramenta para adicionar um novo comentário a uma tarefa no ClickUp com base nos parâmetros fornecidos.
    - Criar Comentário:
        Invocar: "CreateTaskCommentTool" com os parâmetros apropriados.
    """
    args_schema: Type[BaseModel] = CreateTaskCommentRequest
    headers: dict = {"Authorization": f"Bearer {CLICKUP_TOKEN}"}

    def __init__(self, **data):
        super().__init__(**data)

    def _run(self, task_id: str, **comment_data) -> Any:
        """Executa a criação de comentário na tarefa no ClickUp"""

        action = CreateTaskComment()

        url = f"{action.url}{action.path}".format(task_id=task_id)
        params = {key: value for key, value in comment_data.items() if value is not None}

        response = requests.post(url, headers=self.headers, json=params)

        if response.status_code == 200:
            response_json = response.json()
        else:
            try:
                response_json = response.json()
            except requests.JSONDecodeError:
                response_json = {"error": "Invalid JSON response"}
        
        print(f"Response Body: {response_json}")
        return CreateTaskCommentResponse(data=response_json)


## Get Task Comments

In [7]:
from composio.tools.local.clickup.actions.get_task_comments import GetTaskComments, GetTaskCommentsRequest, GetTaskCommentsResponse

class GetTaskCommentsTool(BaseTool):
    name: str = "get_task_comments_tool"
    description: str = """
    Ferramenta para obter os comentários de uma tarefa no ClickUp com base nos parâmetros fornecidos.
    - Obter Comentários:
        Invocar: "GetTaskCommentsTool" com os parâmetros apropriados.
    """
    args_schema: Type[BaseModel] = GetTaskCommentsRequest
    headers: dict = {"Authorization": f"{CLICKUP_TOKEN}"}

    def __init__(self, **data):
        super().__init__(**data)

    def _run(self, task_id: str, **query_params) -> Any:
        """Executa a consulta de comentários da tarefa no ClickUp"""

        action = GetTaskComments()

        url = f"{action.url}{action.path}".format(task_id=task_id)
        params = {key: value for key, value in query_params.items() if value is not None}

        response = requests.get(url, headers=self.headers, params=params)

        if response.status_code == 200:
            response_json = response.json()
        else:
            try:
                response_json = response.json()
            except requests.JSONDecodeError:
                response_json = {"error": "Invalid JSON response"}
        
        print(f"Response Body: {response_json}")
        return GetTaskCommentsResponse(data=response_json)

## Add Task to List

**Necessita do plano ilimitado da ClickUp**

In [8]:
from composio.tools.local.clickup.actions.add_task_to_list import AddTaskToList, AddTaskToListRequest, AddTaskToListResponse

class AddTaskToListTool(BaseTool):
    name: str = "add_task_to_list_tool"
    description: str = """
    Ferramenta para adicionar uma tarefa a uma lista no ClickUp com base nos parâmetros fornecidos.
    - Adicionar Tarefa à Lista:
        Invocar: "AddTaskToListTool" com os parâmetros apropriados.
    """
    args_schema: Type[BaseModel] = AddTaskToListRequest
    headers: dict = {"Authorization": f"Bearer {CLICKUP_TOKEN}"}

    def __init__(self, **data):
        super().__init__(**data)

    def _run(self, list_id: int, task_id: str) -> Any:
        """Executa a adição de tarefa a uma lista no ClickUp"""

        action = AddTaskToList()

        url = f"{action.url}{action.path}".format(list_id=list_id, task_id=task_id)
        params = {}  # Sem parâmetros adicionais para esta ação

        response = requests.post(url, headers=self.headers, json=params)

        if response.status_code == 200:
            response_json = response.json()
        else:
            try:
                response_json = response.json()
            except requests.JSONDecodeError:
                response_json = {"error": "Invalid JSON response"}
        
        print(f"Response Body: {response_json}")
        return AddTaskToListResponse(data=response_json)

## Remove Task From List

In [9]:
from composio.tools.local.clickup.actions.remove_task_from_list import RemoveTaskFromList, RemoveTaskFromListRequest, RemoveTaskFromListResponse

class RemoveTaskFromListTool(BaseTool):
    name: str = "remove_task_from_list_tool"
    description: str = """
    Ferramenta para remover uma tarefa de uma lista no ClickUp com base nos parâmetros fornecidos.
    - Remover Tarefa da Lista:
        Invocar: "RemoveTaskFromListTool" com os parâmetros apropriados.
    """
    args_schema: Type[BaseModel] = RemoveTaskFromListRequest
    headers: dict = {"Authorization": f"Bearer {CLICKUP_TOKEN}"}

    def __init__(self, **data):
        super().__init__(**data)

    def _run(self, list_id: int, task_id: str) -> Any:
        """Executa a remoção de uma tarefa de uma lista no ClickUp"""

        action = RemoveTaskFromList()

        url = f"{action.url}{action.path}".format(list_id=list_id, task_id=task_id)

        response = requests.delete(url, headers=self.headers)

        if response.status_code == 200:
            response_json = response.json()
        else:
            try:
                response_json = response.json()
            except requests.JSONDecodeError:
                response_json = {"error": "Invalid JSON response"}
        
        print(f"Response Body: {response_json}")
        return RemoveTaskFromListResponse(data=response_json)

## Create Folderless List

In [10]:
from composio.tools.local.clickup.actions.create_folderless_list import CreateFolderlessList, CreateFolderlessListRequest, CreateFolderlessListResponse

class CreateFolderlessListTool(BaseTool):
    name: str = "create_folderless_list_tool"
    description: str = """
    Ferramenta para criar uma nova lista em um espaço no ClickUp com base nos parâmetros fornecidos.
    - Criar Lista Sem Pasta:
        Invocar: "CreateFolderlessListTool" com os parâmetros apropriados.
    """
    args_schema: Type[BaseModel] = CreateFolderlessListRequest
    headers: dict = {"Authorization": f"Bearer {CLICKUP_TOKEN}"}

    def __init__(self, **data):
        super().__init__(**data)

    def _run(self, space_id: int, **list_data) -> Any:
        """Executa a criação de uma lista sem pasta no ClickUp"""

        action = CreateFolderlessList()

        url = f"{action.url}{action.path}".format(space_id=space_id)
        params = {key: value for key, value in list_data.items() if value is not None}

        response = requests.post(url, headers=self.headers, json=params)

        if response.status_code == 200:
            response_json = response.json()
        else:
            try:
                response_json = response.json()
            except requests.JSONDecodeError:
                response_json = {"error": "Invalid JSON response"}
        
        print(f"Response Body: {response_json}")
        return CreateFolderlessListResponse(data=response_json)

## Get Lists

In [11]:
from composio.tools.local.clickup.actions.get_lists import GetLists, GetListsRequest, GetListsResponse

class GetListsTool(BaseTool):
    name: str = "get_lists_tool"
    description: str = """
    Ferramenta para visualizar as listas dentro de uma pasta no ClickUp com base nos parâmetros fornecidos.
    - Obter Listas:
        Invocar: "GetListsTool" com os parâmetros apropriados.
    """
    args_schema: Type[BaseModel] = GetListsRequest
    headers: dict = {"Authorization": f"Bearer {CLICKUP_TOKEN}"}

    def __init__(self, **data):
        super().__init__(**data)

    def _run(self, folder_id: int, **query_params) -> Any:
        """Executa a obtenção de listas dentro de uma pasta no ClickUp"""

        action = GetLists()

        url = f"{action.url}{action.path}".format(folder_id=folder_id)
        params = {key: value for key, value in query_params.items() if value is not None}

        response = requests.get(url, headers=self.headers, params=params)

        if response.status_code == 200:
            response_json = response.json()
        else:
            try:
                response_json = response.json()
            except requests.JSONDecodeError:
                response_json = {"error": "Invalid JSON response"}
        
        print(f"Response Body: {response_json}")
        return GetListsResponse(data=response_json)

# Add Dependency

In [12]:
from composio.tools.local.clickup.actions.add_dependency import AddDependency, AddDependencyRequest, AddDependencyResponse

class AddDependencyTool(BaseTool):
    name: str = "add_dependency_tool"
    description: str = """
    Ferramenta para definir uma tarefa como dependente ou bloqueadora de outra tarefa no ClickUp.
    - Adicionar Dependência:
        Invocar: "AddDependencyTool" com os parâmetros apropriados.
    """
    args_schema: Type[BaseModel] = AddDependencyRequest
    headers: dict = {"Authorization": f"{CLICKUP_TOKEN}"}

    def _init_(self, **data):
        super()._init_(**data)

    def _run(self, task_id: str, **query_params) -> Any:
        """Executa a adição de uma dependência de tarefa no ClickUp"""

        action = AddDependency()

        url = f"{action.url}{action.path}".format(task_id=task_id)
        params = {key: value for key, value in query_params.items() if value is not None}
        request_body = {key: query_params.get(key) for key in action.request_params.keys()}

        response = requests.post(url, headers=self.headers, params=params, json=request_body)

        if response.status_code == 200:
            response_json = response.json()
        else:
            try:
                response_json = response.json()
            except requests.JSONDecodeError:
                response_json = {"error": "Invalid JSON response"}
        
        print(f"Response Body: {response_json}")
        return AddDependencyResponse(data=response_json)

## Agente

In [13]:
from langchain import hub
from langchain_openai import AzureChatOpenAI
from langchain.agents import AgentExecutor, create_tool_calling_agent

load_dotenv()

llm = AzureChatOpenAI(model="gpt-4o")
prompt = hub.pull("hwchase17/openai-tools-agent")

In [14]:
prompt_template = f""""\n
Você é Ana Beatriz, secretária e Assistente de Projetos da Cogmo Technology

Using Tools:
You have access to a variety of tools to help you in your conversations.
Make sure to always pass the correct arguments to each tool.

Project Management:
Use ClickUp to manage projects and tasks. To do that, you need to understand its structure.
Users will refer to lists by name (not always the exact name), so you need to search for the closest match and get its ID.
-> Workspace: The highest level of organization in ClickUp. It contains all of your Spaces.
--> Space: A collection of Folders and Lists. It's a way to group related work together.
---> Folder: Used to group Lists together.
----> List: Used to group tasks together.
-----> Task: The basic unit of work in ClickUp. They can be assigned to people, have due dates, and more.
Important ClickUp IDs (Always use these IDs when interacting with ClickUp, unless otherwise specified):
- Your ClickUp user details:
 - id: 84141406
 - username: Ana Beatriz
 - email: ti@cogmo.com.br
- Cogmo Workspace (aka "team_id"): {os.getenv('CLICKUP_TEAM_ID', 12927880)}
- 'Projetos' Space id {os.getenv('CLICKUP_PROJETOS_SPACE_ID', 54804921)}
- "Agentes Folder id: 90131663060
- "clickup_agent" List id: 901304909413
"""

In [15]:
prompt.messages[0].prompt.template = prompt_template

In [16]:
from pprint import pprint

pprint(prompt)

ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, partial_variables={'chat_history': []}, metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'openai-tools-agent', 'lc_hub_commit_hash': 'c18672812789a3b9697656dd539edf0120285dcae36396d0b548ae42a4ed66f5'}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(in

In [17]:
from composio_langchain import ComposioToolSet, Action
from langchain.tools import StructuredTool
from langchain_core.tools import ToolException
from pydantic import ValidationError
from typing import List

def try_except_tool(func):
    try:
        return func
    except ValidationError as e:
        raise ValidationError(f"Validation Error: {e}")    
    except Exception as e:
        raise ToolException(f"Error: {e}")

def enhance_agent_tools(agent_tools: List):
    enhanced_agent_tools = [
        StructuredTool(
            name=tool.name,
            description=tool.description,
            args_schema=tool.args_schema,
            func=try_except_tool(tool.func),
            handle_tool_error=True,
            handle_validation_error=True
        ) for tool in agent_tools
    ]
    return enhanced_agent_tools

def initialize_composio_clickup_tools():
    composio_toolset = ComposioToolSet(api_key=os.getenv('COMPOSIO_API_KEY'))

    selected_clickup_actions = [
            "CLICKUP_AUTHORIZATION_GET_WORK_SPACE_LIST",
            "CLICKUP_AUTHORIZATION_VIEW_ACCOUNT_DETAILS",
            "CLICKUP_FOLDERS_GET_CONTENTS_OF",
            "CLICKUP_FOLDERS_GET_FOLDER_CONTENT",
            "CLICKUP_LISTS_GET_LIST_DETAILS",
            "CLICKUP_MEMBERS_GET_LIST_USERS",
            "CLICKUP_MEMBERS_GET_TASK_ACCESS",
            "CLICKUP_SPACES_GET_DETAILS",
            "CLICKUP_SPACES_GET_SPACE_DETAILS",
            "CLICKUP_TASKS_FILTER_TEAM_TASKS",
            "CLICKUP_TASKS_GET_LIST_TASKS",
            "CLICKUP_TASKS_GET_TASK_DETAILS",
            "CLICKUP_TEAMS_WORK_SPACES_GET_WORK_SPACE_SEATS",
        ]

    clickup_tools = []

    for action in selected_clickup_actions:
        try:
            clickup_tools.extend(composio_toolset.get_actions(actions=[getattr(Action, action)]))
        except Exception as e:
            print(f"Error importing action {action}: {e}")
    clickup_enhanced_tools = enhance_agent_tools(clickup_tools)
    return clickup_enhanced_tools

In [18]:
tools = [
    CreateTaskTool(), 
    DeleteTaskTool(), 
    UpdateTaskTool(), 
    CreateTaskCommentTool(),
    GetTaskCommentsTool(),
    # AddTaskToListTool(),
    RemoveTaskFromListTool(),
    CreateFolderlessListTool(),
    GetListsTool(),
    AddDependencyTool(),
    *initialize_composio_clickup_tools(),
]

agent_memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, memory=agent_memory)

/var/folders/gj/5sh3dt_j0077hpjs7rt7hlrh0000gn/T/ipykernel_17888/4025261537.py:51: DeprecationWarning: Use CLICKUP_GET_FOLDERS instead.
  clickup_tools.extend(composio_toolset.get_actions(actions=[getattr(Action, action)]))
/var/folders/gj/5sh3dt_j0077hpjs7rt7hlrh0000gn/T/ipykernel_17888/4025261537.py:51: DeprecationWarning: Use CLICKUP_GET_FOLDER instead.
  clickup_tools.extend(composio_toolset.get_actions(actions=[getattr(Action, action)]))


Error importing action CLICKUP_LISTS_GET_LIST_DETAILS: type object 'Action' has no attribute 'CLICKUP_LISTS_GET_LIST_DETAILS'
Error importing action CLICKUP_MEMBERS_GET_TASK_ACCESS: type object 'Action' has no attribute 'CLICKUP_MEMBERS_GET_TASK_ACCESS'


/var/folders/gj/5sh3dt_j0077hpjs7rt7hlrh0000gn/T/ipykernel_17888/4025261537.py:51: DeprecationWarning: Use CLICKUP_GET_LIST_MEMBERS instead.
  clickup_tools.extend(composio_toolset.get_actions(actions=[getattr(Action, action)]))
/var/folders/gj/5sh3dt_j0077hpjs7rt7hlrh0000gn/T/ipykernel_17888/4025261537.py:51: DeprecationWarning: Use CLICKUP_GET_SPACE instead.
  clickup_tools.extend(composio_toolset.get_actions(actions=[getattr(Action, action)]))
/var/folders/gj/5sh3dt_j0077hpjs7rt7hlrh0000gn/T/ipykernel_17888/4025261537.py:51: DeprecationWarning: Use CLICKUP_GET_SPACES instead.
  clickup_tools.extend(composio_toolset.get_actions(actions=[getattr(Action, action)]))
/var/folders/gj/5sh3dt_j0077hpjs7rt7hlrh0000gn/T/ipykernel_17888/4025261537.py:51: DeprecationWarning: Use CLICKUP_GET_TASKS instead.
  clickup_tools.extend(composio_toolset.get_actions(actions=[getattr(Action, action)]))


Error importing action CLICKUP_TASKS_FILTER_TEAM_TASKS: type object 'Action' has no attribute 'CLICKUP_TASKS_FILTER_TEAM_TASKS'


/Users/morossini/anaconda3/envs/secretary/lib/python3.11/site-packages/pydantic/json_schema.py:2179: PydanticJsonSchemaWarning: Default value alias='assignees__add' alias_priority=2 description='' extra={} is not JSON serializable; excluding default from JSON schema [non-serializable-default]
  warnings.warn(message, PydanticJsonSchemaWarning)
/Users/morossini/anaconda3/envs/secretary/lib/python3.11/site-packages/pydantic/json_schema.py:2179: PydanticJsonSchemaWarning: Default value alias='assignees__rem' alias_priority=2 description='' extra={} is not JSON serializable; excluding default from JSON schema [non-serializable-default]
  warnings.warn(message, PydanticJsonSchemaWarning)


In [21]:
agent_executor.tools

[CreateTaskTool(),
 DeleteTaskTool(),
 UpdateTaskTool(),
 CreateTaskCommentTool(),
 GetTaskCommentsTool(),
 RemoveTaskFromListTool(),
 CreateFolderlessListTool(),
 GetListsTool(),
 AddDependencyTool()]

In [19]:
# agent_executor.invoke({
#     "input": "Crie uma tarefa chamada 'TESTANDO NOVAMENTE', com vencimento de 02/09/2024, sob sua responsabilidade"
# })

In [20]:
# agent_executor.invoke({
#     "input": "Atualize essa tarefa (id 86a4qp55z): Mude o responsabilidade para o Rafa (id 82061927) e coloque em status 'DOING'"
# })